In [21]:
import psycopg2
import pandas as pd

In [22]:
try:
    connection = psycopg2.connect(
        user="skillfactory",
        password="cCkxxLVrDE8EbvjueeMedPKt",
        host="84.201.134.129",
        port="5432",
        database="skillfactory",
    )
    cursor = connection.cursor()
    # Создаем SQL запрос для получения данных по покупкам курсов
    select_Query = "SELECT\
                        ca.user_id,\
                        ci.resource_id,\
                        ca.promo_code_id,\
                        ca.state\
                    FROM final.cart_items ci\
                        JOIN final.carts ca ON ci.cart_id = ca.id\
                        where ci.resource_type = 'Course'\
                    ORDER by 1,2"  
    cursor.execute(select_Query)  
    carts_records = cursor.fetchall()
except (Exception, psycopg2.Error) as error:
    if connection:
        print("Error", error)
finally:
    if connection:
        cursor.close()
        connection.close()

Создаем DataFrame на основе полученных данных

In [23]:
carts_df = pd.DataFrame(carts_records)

Переименовываем колонки

In [24]:
carts_df.columns = ['id_user', 'id_resource', 'promo', 'state']

Подсчет количества клиентов купивших курсы

In [25]:
mask_sucs = carts_df['state']=='successful'
carts_df[mask_sucs]['id_user'].nunique()

49006

Количество существующих курсов

In [26]:
carts_df['id_resource'].nunique()

127

Среднее число купленных курсов на одного клиента

In [27]:
carts_df[mask_sucs].groupby('id_user')['id_resource'].nunique().mean()

1.4370485246704485

Сколько клиентов купили больше одного курса?

In [31]:
count_course_df = carts_df[mask_sucs].groupby('id_user')['id_resource'].nunique().reset_index()
count_course_df[count_course_df['id_resource'] > 1]['id_user'].nunique()

12656